In [1]:
import numpy as np


def get_value_for_period(time, array):
    """
    Given a point in time (in years), returns the appropriate value for the period 
    
    Parameters:
    time: float 
        time in years
    array: np.array 
        list of values for each 6 months
    """
    return array[int(time*2) if int(time*2)<6 else 5] # works


def correlated_path_generator_3d(S0_A, S0_B, S0_C, T, rates, yields, volatilites, corr_matrix, N, Ni):
    """
    Generates Ni random paths for two correlated underlying asset prices S_t1 and S_t2 using a log-normal process.

    Parameters:
    S0_A, S0_B, S0_C : float
        Initial stock prices
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate (annual)
    q_A, q_B, q_C : float
        Dividend yields (annual) for each stock
    sigma_A, sigma_B, sigma_C : float
        Volatilities of the underlying assets
    corr_matrix : float
        Correlation matrix between the three assets
    N : int
        Number of time steps
    Ni : int
        Number of paths to generate

    Returns:
    tuple of numpy.ndarray
        Arrays of shape (Ni, N+1) containing Ni random paths for S_tA, S_tB and S_tC
    """
    dt = T / N
    paths_A = np.zeros((Ni, N+1))
    paths_B = np.zeros((Ni, N+1))
    paths_C = np.zeros((Ni, N+1))

    paths_A[:, 0] = S0_A
    paths_B[:, 0] = S0_B
    paths_C[:, 0] = S0_C
    
    # Cholesky decomposition to generate correlated random variables
    A = np.linalg.cholesky(corr_matrix)
    
    for i in range(1, N+1):
        Z = np.random.normal(0, 1, (Ni, 3))
        Z_corr = Z.dot(A.T)  # Apply the Cholesky matrix to get correlated random variables
        paths_A[:, i] = paths_A[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[0]) - 0.5 * get_value_for_period(i*dt, volatilites[0])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[0]) * Z_corr[:, 0])
        paths_B[:, i] = paths_B[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[1]) - 0.5 * get_value_for_period(i*dt, volatilites[1])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[1]) * Z_corr[:, 0])
        paths_C[:, i] = paths_C[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[2]) - 0.5 * get_value_for_period(i*dt, volatilites[2])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[2]) * Z_corr[:, 0])

    
    return paths_A, paths_B, paths_C


In [2]:
def best_of_relative_change_payoff(S_A, S_B, S_C, SA_0, SB_0, SC_0, Notional, kappa):
    """
    Calculates the payoff of a note based on the "best of" relative performance of two assets,
    adjusted by a relative strike (kappa).

    Parameters:
    S_A, S_B, S_C : numpy.ndarray
        Final stock prices for each path
    SA_0, SB_0, SC_0 : float
        Initial stock prices
    Notional : float
        Notional value representing the dimension of money
    kappa : float
        Relative strike

    Returns:
    numpy.ndarray
        Payoff for each path
    """
    relative_performance_A = S_A / SA_0
    relative_performance_B = S_B / SB_0
    relative_performance_C = S_C / SC_0

    best_of_relative_performance = np.maximum(relative_performance_A, relative_performance_B, relative_performance_C)
    payoff = Notional * np.maximum(best_of_relative_performance - kappa, 0)
    
    return payoff

In [10]:
def price_note_with_relative_change(paths_A, paths_B, paths_C, S0_A, S0_B, S0_C, Notional, kappa, interest_rates, T):
    """
    Calculates the price of a note based on the "best of" relative performance of two assets,
    adjusted by a relative strike (kappa), and discounts it to present value.

    Parameters:
    paths1, paths2 : numpy.ndarray
        Paths of stock prices for each asset
    SA_0, SB_0, SC_0 : float
        Initial stock prices for each asset
    Notional : float
        Notional value representing the dimension of money
    kappa : float
        Relative strike
    r : float
        Risk-free interest rate (annual)
    T : float
        Time to maturity (in years)

    Returns:
    tuple
        Mean price of the note and its standard error
    """
    # Calculate the payoff at maturity for each path using the modified payoff function
    payoffs = best_of_relative_change_payoff(paths_A[:, -1], paths_B[:, -1], paths_C[:, -1], S0_A, S0_B, S0_C, Notional, kappa)
    
    # Discount payoffs to present value
    pv = np.exp(-sum(0.5*interest_rates[:])) * payoffs  # HAS TO BE THE EXPONENT OF THE INTEGRAL OF THE INTEREST RATES
    
    # Calculate mean price and standard error
    mean_price = np.mean(pv)
    std_error = np.std(pv) / np.sqrt(len(pv))
    
    return mean_price, std_error

In [11]:
# Parameters based on pdf
S0_A, S0_B, S0_C = 150, 250, 300  # Initial stock prices
T = 5  # Time to maturity in years
interest_rates = np.array((0.015, 0.0175, 0.02, 0.021, 0.02, 0.019))
volatilities = np.array([[0.20, 0.21, 0.20, 0.19, 0.18, 0.19],  # Stock A
                         [0.22, 0.23, 0.22, 0.21, 0.20, 0.21],  # Stock B
                         [0.25, 0.26, 0.24, 0.23, 0.22, 0.23]]) # Stock C
interest_rates = np.array([0.015, 0.0175, 0.02, 0.021, 0.02, 0.019])
dividend_yields = np.array([[0.01, 0.015, 0.02, 0.019, 0.018, 0.017],  # Stock A
                            [0.015, 0.02, 0.025, 0.023, 0.021, 0.02],  # Stock B
                            [0.02, 0.025, 0.03, 0.025, 0.023, 0.022]]) # Stock C
r = 0.02  # Risk-free interest rate
q_A, q_B, q_C = 0.015, 0.02, 0.025  # Dividend yields
sigma_A, sigma_B, sigma_C = 0.2, 0.22, 0.25  # Volatilities
corr_matrix = np.array([[1, 0.75, 0.85], [0.75, 1, 0.9], [0.85, 0.9, 1]])
N = 365 * T  # Number of time steps (daily steps)
Ni = 10000  # Number of paths
Notional = 1000000
kappa = 1.05

# Test the function with example parameters
paths_A, paths_B, paths_C = correlated_path_generator_3d(S0_A, S0_B, S0_C, T, interest_rates, dividend_yields, volatilities, corr_matrix, N, Ni)

# Let's check the shape of the generated paths to confirm everything is working as expected
paths_A.shape, paths_B.shape, paths_C.shape

mean_price, std_error = price_note_with_relative_change(paths_A, paths_B, paths_C, S0_A, S0_B, S0_B, Notional, kappa, interest_rates, T)

print(f"Mean Price of the Note: ${mean_price:.2f}")
print(f"Standard Error: ${std_error:.2f}")

Mean Price of the Note: $157413.57
Standard Error: $3279.70
